## 运行准备

### 环境准备

In [2]:
!sh init.sh

Get:1 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease [3611 B]
Hit:2 https://dl.yarnpkg.com/debian stable InRelease                           
Hit:3 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease       
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      m
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1089 kB]
Hit:7 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease          
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]    
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3430 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2942 kB]33m
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1394 k

In [3]:
!pip install .
!pip install numpy

Processing /workspaces/SSG
  Preparing metadata (setup.py) ... done
  Created wheel for pyssg: filename=pyssg-0.1-cp310-cp310-linux_x86_64.whl size=102682 sha256=dad713536b8e2e45fc708aa5248575e9bfd1f20836d89639dd079db5c9484dd0
  Stored in directory: /tmp/pip-ephem-wheel-cache-i13m3zxc/wheels/ec/6e/1e/b6b0a5ab76881d636723508394250141d3535520506f9f2282
Successfully built pyssg
  Attempting uninstall: pyssg
    Found existing installation: pyssg 0.1
    Uninstalling pyssg-0.1:
      Successfully uninstalled pyssg-0.1


### 打包 pyssg

In [6]:
! python setup.py bdist_wheel

running bdist_wheel
running build
running build_ext
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


CMake Deprecation Warning at third_party/pybind11/CMakeLists.txt:8 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


CMake Deprecation Warning at third_party/pybind11/tools/pybind11Tools.cmake:8 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibili

### 准备数据

下载 sift_base.fvecs[huggingface 上，dataset： `qbo-odp/sift1m`]

In [13]:
! git clone http://huggingface.co/datasets/qbo-odp/sift1m

Cloning into 'sift1m'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), 2.03 KiB | 519.00 KiB/s, done.
Filtering content: 100% (4/4), 550.08 MiB | 11.75 MiB/s, done.


### 读取数据

In [ ]:
sift_file = "sift1m/sift_base.fvecs"

In [3]:
import numpy as np
data = np.fromfile(sift_file, dtype=np.float32)
dim = data[0].view(np.int32)
data = data.reshape(-1, dim + 1)
# fvecs 中每行的第0号元素表示每行数据的维数，不是向量数据。
data = np.ascontiguousarray(data[:, 1:]) # 避免内存复制，直接使用连续内存作为数组。
ndata, dim = data.shape

## ssg 测试

In [3]:
import pyssg

pyssg.set_seed(1234)

ImportError: /home/codespace/.python/current/lib/python3.10/site-packages/pyssg.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN8efanna2e10IndexGraphD1Ev

### load fvecs data

In [7]:
pyssg.load_data(sift_file, ndata, dim)

Help on built-in function load_data in module pyssg:

load_data(...) method of builtins.PyCapsule instance
    load_data(arg0: str, arg1: int, arg2: int) -> float
    
    Load data from SIFT-style binary file



In [4]:

index = pyssg.IndexSSG(dim, ndata)


In [ ]:
# 构造 index
index.load("ssg-index", data)

In [ ]:

k, l = 100, 300
query = np.random.randn(dim).astype(np.float32)
knn = index.search(query, k, l)
print(knn)